# 第13章 モナドがいっぱい


```
fmap :: (Functor f) => (a -> b) -> f a -> f b

(<*>) :: (Applicative f) => f (a -> b) -> f a -> f b
```

In [1]:
(*) <$> Just 2 <*> Just 8

Just 16

In [2]:
(++) <$> Just "exdeath" <*> Nothing

Nothing

モナドはある願いを叶えるためのアプリカティブ値の自然な拡張  
「普通の値aを取って、文脈付きの値を返す関数に、文脈付きの値m aを渡したい」という願い  
つまり、a -> m b型の関数をm a型の値に適用したい。ということ

```
(>>=) :: (Monad a) => m a -> (a -> m b) -> m b
```
ここからは `f a` ではなく、`m a` と書くことにする  
`>>=` は bindと呼ばれる  

### Maybeから始めるモナド



In [3]:
fmap (++ "!") (Just "wisdom")

Just "wisdom!"

In [4]:
Just (+3) <*> Just 3

Just 6

`>>=` はモナド値と普通の値を取る関数を引数にとり、なんとかしてその関数をモナド地に適用してモナド値を得る  


In [5]:
(\x -> Just(x+1)) 1

Just 2

In [6]:
applyMaybe :: Maybe a -> (a -> Maybe b) -> Maybe b
applyMaybe Nothing f = Nothing
applyMaybe (Just x) f = f x


In [7]:
Just 3 `applyMaybe` \x -> Just (x + 1)

Just 4

In [8]:
Nothing `applyMaybe` \x -> Just (x + 1)

Nothing

In [9]:
Just 3 `applyMaybe` \x -> if x > 2 then Just x else Nothing

Just 3

In [10]:
Just 1 `applyMaybe` \x -> if x > 2 then Just x else Nothing

Nothing

### Monad型クラス

```
class Monad m where
    return :: a -> m a
    
    (>>=) :: m a -> (a -> m b) -> m b
    
    (>>) :: m a -> m b -> m b
    x >> y = x >> \_ -> y
    
    fail :: String -> m a
    fail msg = error msg
```

`class (Applicative m) = > Monad m where` という型クラス制約はないけど、全てのモナドはアプリカティブファンクター  

```
instance Monad Maybe where
    return x = Just x
    Nothing >>= f = Nothing
    Just x >>= f = f x
    fail _ = Nothing
```


In [13]:
return 1 :: Maybe Int

Just 1

次回

## 綱渡り

デュエルスタンバイ